# Openai Proxy Quickstart
> A simple tool to capture metadata about your interactions with the OpenAI API.  Currently supports ChatCompletion endpoint, including streaming.

## Setup

All you need to do is:
1. Change your openai api key to the format `{wandb_api_key}:{openai_api_key}`; and
2. Change your openai api base url to `https://wandb.ai/proxy/openai/v1`

In [ ]:
import os
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")
wandb_api_key = os.getenv("WANDB_API_KEY")

In [ ]:
openai.api_base = "https://api.wandb.ai/proxy/openai/v1"
openai.api_key = f"{wandb_api_key}:{openai_api_key}"

## Start logging
You can use your existing API calls as-is, now with extra logging!

By default, this will log to `{your_entity}/monitoring/openai`, where the project is `monitoring` and the table name is `openai`.


### Using OpenAI SDK

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Tell me a joke about gradients!"}],
)
response

### Using CURL

In [ ]:
%%bash
curl "https://api.wandb.ai/proxy/openai/v1/chat/completions" \
-H "Authorization: Bearer $WANDB_API_KEY:$OPENAI_API_KEY" \
-H "Content-Type: application/json" \
-d '{
     "model": "gpt-3.5-turbo",
     "messages": [{"role": "user", "content": "Tell me a joke about loss functions!"}]
   }'

## Configuration
Optionally specify headers to determine where you want your data to be created.

| Header | Description | Default |
| --- | --- | --- |
| `X-Wandb-Entity` | W&B entity name | Your default entity |
| `X-Wandb-Project` | W&B project name | `monitoring` |
| `X-Wandb-Stream` | StreamTable name | `openai` |

By default, proxy data will show up in `{your_entity}/monitoring/openai`.

### Advanced configuration
1. Optionally add custom attributes to your table with the `X-Wandb-Attribute-` prefix.  For example, adding the header `X-Wandb-Attribute-Foo: bar` will add a column `Foo` with the value `bar` to your table.
2. Optionally add the `X-Wandb-Client-Id` header if you want to further group related requests together.  By default, each request will have its own unique Client ID.

### Using OpenAI SDK

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Tell me a joke about gradients as if you were Shakespeare"}],
    headers={
        "X-Wandb-Entity": "your-entity",
        "X-Wandb-Project": "your-project",
        "X-Wandb-Stream": "your-stream",
        "X-Wandb-Client-Id": "Shakespearean-Gradients-001",
        "X-Wandb-Attribute-Persona": "Shakespeare",
    }
)
response

### Use with CURL

In [ ]:
%%bash
curl "https://api.wandb.ai/proxy/openai/v1/chat/completions" \
-H "Authorization: Bearer $WANDB_API_KEY:$OPENAI_API_KEY" \
-H "Content-Type: application/json" \
-H "X-Wandb-Entity: your-entity" \
-H "X-Wandb-Project: your-project" \
-H "X-Wandb-Stream: your-stream" \
-H "X-Wandb-Client-Id: Shakespearean-Gradients-001" \
-H "X-Wandb-Attribute-Persona: Shakespeare" \
-d '{
     "model": "gpt-3.5-turbo",
     "messages": [{"role": "user", "content": "Tell me a joke about loss functions!"}]
   }'

## Use Proxy with Weave SDK
You can use this proxy with the Weave SDK to log out to a personal streamtable and a proxy-configured streamtable simultaneously.  This can be useful for admins who want to monitor API calls in their organization.
- Users specify a personal table path with `init_monitor(entity/project/table)`.  Any `ChatCompletion.create` calls with automatically log to this table.
- Then, if the `openai.api_base` and `openai.api_key` are set to proxy format, the above calls will also log the proxy-configured table.

NOTE: This currently does not work with streaming `ChatCompletion`, but we hope to support it soon!

In [ ]:
from weave.monitoring import openai as openaimon
from weave.monitoring import init_monitor

m = init_monitor("personal-entity/personal-project/personal-stream")

openai.api_base = "https://api.wandb.ai/proxy/openai/v1"
openai.api_key = f"{wandb_api_key}:{openai_api_key}"

OPENAI_MODEL = 'gpt-3.5-turbo'

r = openaimon.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": f"hello world!"}])
r = openaimon.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": f"what is 2+2?"}])